# Heroes Of  Pymoli Data Analysis

-  Trend 1: Males play and purchase the most. 
-  Trend 2: Ages 15-29 play and purchase the most. 
-  Trend 3: Most profitable item doesn't always mean it's the most popular. 

In [1]:
# Dependencies
import pandas as pd
import os

# Import the JSON file
file_path = os.path.join("raw_data", "purchase_data.json")

# Read it into pandas dataframe
purchase_data_df = pd.read_json(file_path)
#purchase_data_df.head()

## Player Count

In [2]:
# Find the total number of players
# Use .nunique() over len() since some players may have purchased more than once
players_count = purchase_data_df["SN"].nunique()
#players_count

# Create a DataFrame to organize the data
players_count_df = pd.DataFrame({
    "Total Players": [players_count]
})
players_count_df

,Total Players
0,573


## Purchasing Analysis (Total)

In [3]:
# Find the number of unique items
unique_items = purchase_data_df["Item ID"].nunique()
#unique_items

# Find the average purchase price
average_purchase_price = purchase_data_df["Price"].mean()
#average_purchase_price

# Find the total number of purchases
purchases_count = len(purchase_data_df)
#purchases_count

# Find the total revenue
total_revenue = purchase_data_df["Price"].sum()
#total_revenue

# Create a DataFrame to organize the data
purchasing_analysis_df = pd.DataFrame({
    "Number of Unique Items": [unique_items], 
    "Average Price": [average_purchase_price], 
    "Number of Purchases": [purchases_count], 
    "Total Revenue": [total_revenue]
}, columns=["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"])
#purchasing_analysis_df

# Use mapping to clean the $ columns 
purchasing_analysis_df["Average Price"] = purchasing_analysis_df["Average Price"].map("${:.2f}".format)
purchasing_analysis_df["Total Revenue"] = purchasing_analysis_df["Total Revenue"].map("${:.2f}".format)
purchasing_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$2.93,780,$2286.33


## Gender Demographics

In [4]:
# Need to single out the players since the same player can purchase multiple times
organized_purchase_data = purchase_data_df[["Age", "Gender", "SN"]]
purchase_data_no_duplicates = organized_purchase_data.drop_duplicates()
#purchase_data_no_duplicates.count()

# Can groupby by Gender to calculate just that column 
# Find the count for each gender (male, female, other/non-disclosed)
gender_groupby = purchase_data_no_duplicates.groupby("Gender")
gender_count = gender_groupby["Gender"].count()
#gender_count

# Fid the percentage count for each gender
gender_percentage = gender_groupby["Gender"].count() / players_count * 100
#gender_percentage

# Create a DataFrame to organize the data
# Sort index so male is up top
gender_demographics = pd.DataFrame({
    "Total Count": gender_count, 
    "Percentage of Players": gender_percentage
}).sort_values(["Total Count"], ascending=False)
#gender_demographics

# Use mapping to clean the % columns 
gender_demographics["Percentage of Players"] = gender_demographics["Percentage of Players"].map("{:.2f}%".format)
gender_demographics

,Percentage of Players,Total Count
Gender,,
Male,81.15%,465
Female,17.45%,100
Other / Non-Disclosed,1.40%,8


## Purchasing Analysis (Gender)

In [5]:
# Use the original data since we're looking at all purchases
# Can groupby by Gender to calculate just that column 
gender_purchase_groupby = purchase_data_df.groupby("Gender")

# Find the count for each gender (male, female, other/non-disclosed)
gender_purchase_count = gender_purchase_groupby["Gender"].count()
#gender_purchase_count

# Find the average purchase price for each gender
average_purchase_gender = gender_purchase_groupby["Price"].mean()
#average_purchase_gender

# Find the total purchase amount for each gender
total_purchase_gender = gender_purchase_groupby["Price"].sum()
#total_purchase_gender

# Find the normalized totals for each gender
normalized_total_gender = (total_purchase_gender / gender_purchase_count) 
normalized_total_gender

# Create a DataFrame to organize the data
# Sort index so male is up top
purchasing_gender_df = pd.DataFrame({ 
    "Purchase Count": gender_purchase_count, 
    "Average Purchase Price": average_purchase_gender, 
    "Total Purchase Value": total_purchase_gender, 
    "Normalized Totals": normalized_total_gender
},
columns=["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]).sort_values(["Purchase Count"], ascending=False)
#purchasing_gender_df

# Use mapping to clean the $ columns 
purchasing_gender_df["Average Purchase Price"] = purchasing_gender_df["Average Purchase Price"].map("${:.2f}".format)
purchasing_gender_df["Total Purchase Value"] = purchasing_gender_df["Total Purchase Value"].map("${:.2f}".format)
purchasing_gender_df["Normalized Totals"] = purchasing_gender_df["Normalized Totals"].map("${:.2f}".format)
purchasing_gender_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Male,633,$2.95,$1867.68,$2.95
Female,136,$2.82,$382.91,$2.82
Other / Non-Disclosed,11,$3.25,$35.74,$3.25


## Age Demographics

In [6]:
# Need to use the data with no duplicates so players aren't counted more than once
#purchase_data_no_duplicates.count()

# Need to create bins for the age groups first
bins = [0, 10, 15, 20, 25, 30, 35, 40, 100]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
age_bins = pd.cut(purchase_data_no_duplicates["Age"], bins, labels=group_names, right=False)
#age_bins

# Find the count for each age group
age_count = age_bins.value_counts()
#age_count

# Find the percentage for each age group 
age_percent = age_count / players_count * 100
#age_percent

# Create DataFrames to organize the data
# Need to sort by the indexes or the #s will be greatest to least based on count
age_demographics = pd.DataFrame({
    "Total Counts": age_count, 
    "Percentage of Players": age_percent
}).sort_index()
#age_demographics

# Use mapping to clean the % columns 
age_demographics["Percentage of Players"] = age_demographics["Percentage of Players"].map("{:.2f}%".format)
age_demographics

,Percentage of Players,Total Counts
<10,3.32%,19
10-14,4.01%,23
15-19,17.45%,100
20-24,45.20%,259
25-29,15.18%,87
30-34,8.20%,47
35-39,4.71%,27
40+,1.92%,11


## Purchasing Analysis (Age)

In [7]:
# Use from original data since we're looking at all purchases 
# Need to create bins for the age groups first (original data)
bins2 = [0, 10, 15, 20, 25, 30, 35, 40, 100]
group_names2 = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
age_bins2 = pd.cut(purchase_data_df["Age"], bins, labels=group_names, right=False)
#age_bins2

# Make a copy of the original data first to not override the original
purchase_data_copy = purchase_data_df.copy()

# Replace the age column by the bins made to filter and calculate
purchase_data_copy["Age"] = age_bins2
#purchase_data_copy.head()

# Can groupby by Age to calculate just that column 
# Find the count for each age group 
age_purchase_groupby = purchase_data_copy.groupby("Age")
age_purchase_count = age_purchase_groupby["Gender"].count()
#age_purchase_count

# Find the average purchase price for each age group
average_purchase_age = age_purchase_groupby["Price"].mean()
#average_purchase_age

# Find the total purchase amount for each age group
total_purchase_age = age_purchase_groupby["Price"].sum()
#total_purchase_age

# Find the normalized totals for each age group
normalized_total_age = (total_purchase_age / age_purchase_count) 
#normalized_total_age

# Create a DataFrame to organize the data
purchasing_age_df = pd.DataFrame({ 
    "Purchase Count": age_purchase_count, 
    "Average Purchase Price": average_purchase_age, 
    "Total Purchase Value": total_purchase_age, 
    "Normalized Totals": normalized_total_age
},
columns=["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"])
#purchasing_age_df

# Use mapping to clean the $ columns 
purchasing_age_df["Average Purchase Price"] = purchasing_age_df["Average Purchase Price"].map("${:.2f}".format)
purchasing_age_df["Total Purchase Value"] = purchasing_age_df["Total Purchase Value"].map("${:.2f}".format)
purchasing_age_df["Normalized Totals"] = purchasing_age_df["Normalized Totals"].map("${:.2f}".format)
purchasing_age_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age,,,,
<10,28,$2.98,$83.46,$2.98
10-14,35,$2.77,$96.95,$2.77
15-19,133,$2.91,$386.42,$2.91
20-24,336,$2.91,$978.77,$2.91
25-29,125,$2.96,$370.33,$2.96
30-34,64,$3.08,$197.25,$3.08
35-39,42,$2.84,$119.40,$2.84
40+,17,$3.16,$53.75,$3.16


## Top Spenders

In [8]:
# Use groupby to separate the data according to "SN" values
groupby_sn = purchase_data_df.groupby(["SN"])

# Need to use a data function to view the groupby object
#groupby_sn.count().head()

# Get the purchase count, average purchase price, and total purchase value 
sn_price_count = groupby_sn["Price"].count()
#print(sn_price_count.head())
sn_price_mean = groupby_sn["Price"].mean()
#print(sn_price_mean.head())
sn_price_sum = groupby_sn["Price"].sum()
#print(sn_price_sum.head())

top_spenders = pd.DataFrame({
    "Purchase Count": sn_price_count, 
    "Average Purchase Price":sn_price_mean, 
    "Total Purchase Value": sn_price_sum
},
columns=["Purchase Count", "Average Purchase Price", "Total Purchase Value"])

top_spenders = top_spenders.sort_values(["Total Purchase Value"], ascending=False).head().round(2)

top_spenders["Average Purchase Price"] = top_spenders["Average Purchase Price"].map("${:.2f}".format)
top_spenders["Total Purchase Value"] = top_spenders["Total Purchase Value"].map("${:.2f}".format)

top_spenders.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


## Most Popular Items

In [9]:
# Use groupby to separate the data according to "Item ID" and "Item Name" values
groupby_item = purchase_data_df.groupby(["Item ID", "Item Name"])

# Need to use a data function to view the groupby object
#groupby_item.count().head()

# Get the purchase count, item price, and total purchase value 
item_price_count = groupby_item["Price"].count()
#print(item_price_count.head())
item_price = groupby_item["Price"].mean()
#print(item_price.head())
item_price_sum = groupby_item["Price"].sum()
#print(item_price_sum.head())

# Create a DataFrame to organize the data
most_popular = pd.DataFrame({
    "Purchase Count": item_price_count, 
    "Item Price":item_price, 
    "Total Purchase Value": item_price_sum
},
columns=["Purchase Count", "Item Price", "Total Purchase Value"])

# Sort by purchase count in descending order to see the top 5 items that were the most purchased
most_popular = most_popular.sort_values(["Purchase Count"], ascending=False)

# Using mapping to format columns in $ 
most_popular["Item Price"] = most_popular["Item Price"].map("${:.2f}".format)
most_popular["Total Purchase Value"] = most_popular["Total Purchase Value"].map("${:.2f}".format)

most_popular.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41


## Most Profitable Items

In [10]:
# Create a DataFrame to organize the data
most_profitable = pd.DataFrame({
    "Purchase Count": item_price_count, 
    "Item Price":item_price, 
    "Total Purchase Value": item_price_sum
},
columns=["Purchase Count", "Item Price", "Total Purchase Value"])

# Sort by purchase count in descending order to see the top 5 items that were the most purchased
most_profitable = most_profitable.sort_values(["Total Purchase Value"], ascending=False)

# Using mapping to format columns in $ 
most_profitable["Item Price"] = most_profitable["Item Price"].map("${:.2f}".format)
most_profitable["Total Purchase Value"] = most_profitable["Total Purchase Value"].map("${:.2f}".format)

most_profitable.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
